We implement the algorithm of Trimborg, Koch, and Steger (2008) to solve a particular linear problem $J X = E$, where $J$ is an $NM\times NM$ matrix and $E$ is a vector of length $NM$. 
$J$ has the following form:
$$J=\begin{pmatrix}
J^0_1 & 0 & ... & 0 & 0 \\ 
J^1_1 & J^1_2 & & ... 0 & 0 \\ 
0 & 0 & ... & J^{M-1}_{M-1} & J^{M-1}_M \\
0 & 0 & ... & 0 & J^M_M 
\end{pmatrix},$$
with $N=N_1+N_2$. 
$J^0_1 = (I_{N_1xN_1}\ 0_{N_1xN_2})$ of dimension $N_1\times N$, $J^M_M =(0_{N_2xN_1}\ I_{N_2xN_2})$ of dimension $N_2\times N$, and the remaining matrices have dimension $N\times N$.

$N_1$ is the number of initial conditions and $N_2$ the number of final conditions.
$J^0_1 X_1=E_1$ is the system of initial conditions, and $J^M_M X_M=E_M$ is the system of final conditions.


The key to avoid numerical instability or computational complexity is to find a way to solve efficiently this linear system without inverting the (huge) matrix $J$. The strategy consists in "triangularizing" the matrix block by block. We combine two algebraic operations.

## Operation 1:
Simplify the system
$\begin{pmatrix}
A_{n,n} & B_{n,m} & 0_{n,p} \\ 
C_{q,n} & D_{q,m} & F_{q,p} \\ 
0_{r,n} & G_{r,m} & H_{r,p} 
\end{pmatrix}X =\begin{pmatrix}E^1_n \\ E^2_q \\ E^3_p\end{pmatrix}$
into
$\begin{pmatrix}
U_{n,n} & L^{-1}PB_{n,m} & 0_{n,p} \\ 
C_{q,n} & D_{q,m}  & F_{q,p} \\ 
0_{r,n} & G_{r,m} & H_{r,p} 
\end{pmatrix}X =\begin{pmatrix}L^{-1}P E^1_n \\ E^2_q  \\ E^3_p\end{pmatrix}$
by pre-multiplying by
$\begin{pmatrix}
L^{-1}P_{n,n} & 0_{n,q} & 0_{n,r} \\ 
0_{q,n} & I_{q,q} & 0_{q,r} \\ 
0_{r,n} & 0_{r,q} & I_{r,r} 
\end{pmatrix}$ with the PLU decomposition $A=P^{-1}LU$. This only affects the first row of the system.

In [1]:
function operation1!(J, E, index, n, m)
    ## J is large matrix, E large vector
    ## index+1 is the first element of matrix A
    A = @view J[index+1 : index+n, index+1 : index+n]
    B = @view J[index+1 : index+n, index+n+1 : index+n+m]
    E1 = @view E[index+1 : index+n]
    
    facto = lu(A)
    aux = inv(facto.L) * facto.P ## compute matric L-1 P
    A[:] = facto.U 
    B[:] = aux * B 
    E1[:] = aux * E1
    return()
end

operation1! (generic function with 1 method)

## Operation 2:
Simplify the system
$\begin{pmatrix}
U_{n,n} & B_{n,m} & 0_{n,p} \\ 
C_{q,n} & D_{q,m} & F_{q,p} \\ 
0_{r,n} & G_{r,m} & H_{r,p} 
\end{pmatrix}X =\begin{pmatrix}E^1_n \\ E^2_q \\ E^3_p\end{pmatrix}$
into
$\begin{pmatrix}
U_{n,n} & B_{n,m} & 0_{n,p} \\ 
0_{q,n} & D-CU^{-1}B  & F_{q,p} \\ 
0_{r,n} & G_{r,m} & H_{r,p} 
\end{pmatrix}X =\begin{pmatrix}E^1_n \\ E^2-CU^{-1}E^1  \\ E^3_p\end{pmatrix}$
by pre-multiplying by
$\begin{pmatrix}
I_{n,n} & 0_{n,q} & 0_{n,r} \\ 
-C_{q,n}U^{-1}_{n,n} & I_{q,q} & 0_{q,r} \\ 
0_{r,n} & 0_{r,q} & I_{r,r} 
\end{pmatrix}.$
Matrices $F$, $G$ and $H$ remain unchanged. We only work with the block matrix $UBCD$. This only affects the second raw of the system.

In [2]:
function operation2!(J, E, index, n, m, q)
    ## J is large matrix, E large vector
    ## index+1 is the first element of matrix U
    U = @view J[index+1 : index+n, index+1 : index+n]
    B = @view J[index+1 : index+n, index+n+1 : index+n+m]
    E1 = @view E[index+1 : index+n]
    C = @view J[index+n+1 : index+n+q, index+1 : index+n]
    D = @view J[index+n+1 : index+n+q, index+n+1 : index+n+m]
    E2 = @view E[index+n+1 : index+n+q]
        
    aux = C * inv(U)
    C[:] = fill(0., size(C))
    D[:] = D - aux * B
    E2[:] = E2 - aux * E1
    return()
end

operation2! (generic function with 1 method)

The algorithm is the following. 
1) Start with operation 2 on the first $N_1$ columns of the block $(N_1+N)\times (N_1+N_2)$.
2) For $i$ from 1 to $M-2$, apply operation 1 on blocks $N \times (N+N_2)$, and then operation 2 on blocks $(N_1+N)\times (N_1+N_2)$.
3) Lastly, use operation 1 to diagonalize the penultimate block.

In [3]:
function TKSsolve(J, E, N1, N2, M)
    ## J is large matrix, E large vector
    N = N1+N2
    
    ## initial step
    operation2!(J, E, 0, N1, N2, N)
   
    ## loop
    for i in 1:(M-2)
        ##op1
        index = N1 + N*(i-1)
        operation1!(J, E, index, N, N2)
    
        ##op2
        index = index+N2
        operation2!(J, E, index, N1, N2, N)
    end
   
    ## final step
    index = N1 + N*(M-2)
    operation1!(J, E, index, N, N2)
   
    return(J \ E)
end

TKSsolve (generic function with 1 method)

To check the algorithm and precompile, we apply the algorithm on a random matrix.